In [170]:
import pandas as pd
import numpy as np
import sys
from pandas import HDFStore
import matplotlib.pyplot as pl
import lasagne as ls
from theano import tensor as T
from lasagne.layers import InputLayer, DenseLayer
from lasagne.updates import nesterov_momentum
from lasagne.nonlinearities import rectify
from nolearn.lasagne import NeuralNet
from ConfigParser import SafeConfigParser
store = HDFStore("storeTraffic.h5")
#"ita_public_tools/output/data.csv"
data = pd.Series.from_csv("10min_workload.csv",header=None,index_col=None)
def read_config():
    parser = SafeConfigParser()
    parser.read('configNeural.cfg')
    hidden_layer = int(parser.get("Neural","hidden_layer"))
    epochs = int(parser.get("Neural","epochs"))
    return hidden_layer, epochs
class LoadParam():
    def initNN(self,n_input):
        #Build layer for MLP
        hidden_layer, epochs = read_config()
        l_in = ls.layers.InputLayer(shape=(None,n_input),input_var=None)
        l_hidden = ls.layers.DenseLayer(l_in,num_units=hidden_layer,nonlinearity=ls.nonlinearities.rectify)
        network = l_out = ls.layers.DenseLayer(l_hidden,num_units=1)
        print "Neural network initialize"
        #Init Neural net
        net1 = NeuralNet(
            layers=network,
            # optimization method:
            update=nesterov_momentum,
            update_learning_rate=0.000001,
            update_momentum=0.9,
            regression=True,  # flag to indicate we're dealing with regression problem
            max_epochs=epochs,  # we want to train this many epochs
            verbose=1,
        )
        return net1
    def __init__(self,n_type,n_input,n_periodic=0):
        self.n_input = n_input
        self.n_periodic = n_periodic
        if(n_periodic==0):
            self.net = self.initNN(n_input)
            if(n_type=="NN"):
                self.net.load_params_from('Params/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
            elif(n_type=="GN"):
                self.net.load_params_from('GeneticParams/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
        else:
            self.net = self.initNN(n_input+n_periodic)
            if(n_type=="NN"):
                self.net.load_params_from('ParamsPeriodic/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
            elif(n_type=="GN"):
                self.net.load_params_from('GeneticParamsPeriodic/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
    def normalize(self,dataCount,dataTest):
        dataNorm = pd.Series(np.zeros(dataCount.shape[0]),dtype=np.float64)
        dataNorm = (dataCount - dataTest.min())/(dataTest.max()-dataTest.min())
        return dataNorm
    def normalize(self,dataCount):
        dataNorm = pd.Series(np.zeros(dataCount.shape[0]),dtype=np.float64)
        dataNorm = (dataCount - dataCount.min())/(dataCount.max()-dataCount.min())
        return dataNorm
    def convert(self,data,dataTest):
        max = dataTest.max()
        min = dataTest.min()
        return (data*(max-min)+min)
    def generate(self,range_training,range_test=1):
        # In[62]:
        print "Loading storage"
        print "generate data"
        data_training = self.normalize(data[142*range_training[0]-self.n_input:142*range_training[1]])
        X_training = self.getTraining(data_training)
#         data_validation = data[142*range_training[1]-self.n_input:142*(range_training+range_test)]
        data_test = self.normalize(data[142*range_training[0]:142*range_training[1]])
        return X_training,data_test
    def getTraining(self,data_training):
        n_row = data_training.shape[0]
        X_training = []
        for t in range(self.n_input,n_row):
            temp = []
            for i in range(0,self.n_input):
                temp.append(data_training.iloc[t-i-1])
            for j in range(1,self.n_periodic+1):
                start_idx = data.index[t]
                temp.append(data[start_idx-142*j])
            X_training.append(temp)
        return np.asarray(X_training)
    def predict(self,X_test):
#         dataTest= pd.read_sql("SELECT count FROM workload where time >= 895096802-%d and time < 895096802+86400"%(n_input),conn)["count"]
        return self.net.predict(X_test)
    def score(self,X_test,y_actual):
        return self.net.score(X_test,y_actual)

In [171]:
nn = LoadParam("GN",18)

Neural network initialize
Loaded parameters to layer 'dense1' (shape 20x15).
Loaded parameters to layer 'dense1' (shape 15).
Loaded parameters to layer 'dense2' (shape 15x1).
Loaded parameters to layer 'dense2' (shape 1).


In [172]:
X_test,y_test = nn.generate((3,5))

Loading storage
generate data


In [173]:
print nn.score(X_test,y_test)

0.00583827844558


In [174]:
gn_pred = nn.predict(X_test)
ax = pl.subplot()
ax.set_color_cycle(['blue','red'])
ax.plot(gn_pred,label="Genetic Neural Network")
ax.set_title("Genetic Neural Network")
ax.plot(y_test,label="Actual")
ax.legend()
# ax1.plot(GN.y_training)
# ax1.legend(["Predict","Actual"])
# # ax1.set_color_cycle(['blue','red'])
# ax2.plot(nn_pred,label="Neural Network")
# ax2.plot(NN.y_training)
# ax2.set_title("Neural Network")
# ax2.set_color_cycle(['blue','red'])
pl.show()

In [59]:
# data = pd.read_csv("10min_workload.csv",names=["Count"])
# lst = (142*30,142*61)
# data_total = data[lst[0]:lst[1]]
# ax = pl.subplot()
# ax.set_color_cycle(['blue','red','green'])
# # ax.plot(gn_pred,label="Genetic Neural Network")
# t = np.arange(0,data_total.shape[0],142)
# ax.plot(data_total,label="Actual")
# for i in np.arange(0,data_total.shape[0],142):
#     ax.axvline(i,color='r')
# ax.legend()
# pl.show()